In [1]:
print("womp womp")

womp womp


In [2]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch.optim as optim
from collections import OrderedDict
from torchvision import datasets
from torchvision import transforms
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import precision_score, recall_score, accuracy_score

%load_ext autotime

time: 0 ns (started: 2023-12-10 21:04:34 -05:00)


In [3]:
file_path = r"C:\Users\aaron\Documents\Python Code\Datasets\Heart Risk Combined Final Dataset.csv"

heart_attack = pd.read_csv(file_path)

heart_attack.head()

,Age,Sex,Blood Pressure,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Risk
0,40,M,140,289,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,49,F,160,180,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,37,M,130,283,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,48,F,138,214,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,54,M,150,195,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


time: 32 ms (started: 2023-12-10 21:04:34 -05:00)


In [4]:
#Defining mapping function to map multiple inputs of the dataset
def gender_map(x):
    return x.map({'M': 1, 'F': 0, 'Male' : 1, "Female" : 0, '1' : 1, '0' : 0})

def hemisphere_map(x):
    return x.map({'Southern Hemisphere' : -1, 'Northern Hemisphere' : 1})

def countries_map(x):
    return x.map({'Argentina' : -1, 'Brazil' : -1, 'China' : -1, 'Colombia' : -1,
    'India' : -1, 'Nigeria' : -1, 'South Africa' : -1, 'South Korea' : -1, 'Thailand' : -1,
    'Vietnam' : -1, 'Australia' : 1, 'Canada' : 1, 'France' : 1, 'Germany' : 1,
    'Italy' : 1, 'Japan' : 1, 'New Zealand' : 1, 'Spain' : 1, 'United Kingdom' : 1,
    'United States' : 1})

def diet_map(x):
    return x.map({'Unhealthy' : -1, 'Average' : 0, 'Healthy' : 1})

def continent_map(x):
    return x.map({'Asia' : 0, 'Africa' : 1, 'Europe' : 2, 'North America' : 3,
    'South America' : 4, 'Australia' : 5})

time: 0 ns (started: 2023-12-10 21:04:34 -05:00)


In [5]:
heart_attack['Sex'] = heart_attack[['Sex']].apply(gender_map)
heart_attack['Hemisphere'] = heart_attack[['Hemisphere']].apply(hemisphere_map)
heart_attack['Country'] = heart_attack[['Country']].apply(countries_map)
heart_attack['Diet'] = heart_attack[['Diet']].apply(diet_map)
heart_attack['Continent'] = heart_attack[['Continent']].apply(continent_map)

unique_sex = heart_attack['Sex'].unique()
print(unique_sex)
unique = heart_attack['Hemisphere'].unique()
print(unique)
unique_country = heart_attack['Country'].unique()
print(unique_country)
unique_diet = heart_attack['Diet'].unique()
print(unique_diet)
unique_continent = heart_attack['Continent'].unique()
print(unique_continent)

heart_attack.head()

[1 0]
[nan -1.  1.]
[nan -1.  1.]
[nan  0. -1.  1.]
[nan  4.  3.  2.  0.  1.  5.]


,Age,Sex,Blood Pressure,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Risk
0,40,1,140,289,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,49,0,160,180,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,37,1,130,283,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,48,0,138,214,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,54,1,150,195,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


time: 15 ms (started: 2023-12-10 21:04:34 -05:00)


In [6]:
mode_columns = ['Stress Level','Medication Use', 'Previous Heart Problems', 'Diet', 'Physical Activity Days Per Week', 'Alcohol Consumption', 'Obesity', 'Smoking', 
                'Family History', 'Diabetes', 'Hemisphere', 'Continent', 'Country']

mean_columns = ['Exercise Hours Per Week', 'Sleep Hours Per Day', 'BMI', 'Sedentary Hours Per Day']

median_columns = ['Heart Rate', 'Income', 'Triglycerides']

# Fill NaN values with the mode of the column for mode_columns
for col in mode_columns:
    heart_attack[col].fillna(heart_attack[col].mode()[0], inplace=True)

# Fill NaN values with the median of the column for median_columns
for col in median_columns:
    heart_attack[col].fillna(heart_attack[col].median(), inplace=True)

# Fill NaN values with the mean of the column for mean_columns
for col in mean_columns:
    heart_attack[col].fillna(heart_attack[col].mean(), inplace=True)


unique_sex = heart_attack['Sex'].unique()
print(unique_sex)
unique = heart_attack['Hemisphere'].unique()
print(unique)
unique_country = heart_attack['Country'].unique()
print(unique_country)
unique_diet = heart_attack['Diet'].unique()
print(unique_diet)
unique_continent = heart_attack['Continent'].unique()
print(unique_continent)


# Check for NaN values in columns after fillna operations
columns_with_nulls = heart_attack.columns[heart_attack.isnull().any()].tolist()
print("Columns with NaN values after fillna operations:")
print(columns_with_nulls)

heart_attack.head()

[1 0]
[ 1. -1.]
[ 1. -1.]
[ 1.  0. -1.]
[0. 4. 3. 2. 1. 5.]
Columns with NaN values after fillna operations:
[]


,Age,Sex,Blood Pressure,Cholesterol,Heart Rate,Diabetes,Family History,Smoking,Obesity,Alcohol Consumption,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Risk
0,40,1,140,289,75.0,1.0,0.0,1.0,1.0,1.0,...,5.99369,157866.0,28.891446,417.0,3.0,7.023508,1.0,0.0,1.0,0
1,49,0,160,180,75.0,1.0,0.0,1.0,1.0,1.0,...,5.99369,157866.0,28.891446,417.0,3.0,7.023508,1.0,0.0,1.0,1
2,37,1,130,283,75.0,1.0,0.0,1.0,1.0,1.0,...,5.99369,157866.0,28.891446,417.0,3.0,7.023508,1.0,0.0,1.0,0
3,48,0,138,214,75.0,1.0,0.0,1.0,1.0,1.0,...,5.99369,157866.0,28.891446,417.0,3.0,7.023508,1.0,0.0,1.0,1
4,54,1,150,195,75.0,1.0,0.0,1.0,1.0,1.0,...,5.99369,157866.0,28.891446,417.0,3.0,7.023508,1.0,0.0,1.0,0


time: 16 ms (started: 2023-12-10 21:04:34 -05:00)


In [7]:
# Check if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")  # Use GPU
    print("GPU is available.")
else:
    device = torch.device("cpu")   # Use CPU
    print("GPU is not available, using CPU.")

GPU is not available, using CPU.
time: 0 ns (started: 2023-12-10 21:04:34 -05:00)


In [8]:
#Splitting the Variables into X and Y
X = heart_attack.iloc[:, 0:23].values
Y = heart_attack.iloc[:, 24].values

# Split our Data set into Training Data and val Data.
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state = 0)

Y_train = Y_train.reshape(-1, 1)
Y_val = Y_val.reshape(-1, 1)

# Convert the numpy arrays into tensors
Y_train_t = torch.tensor(Y_train, dtype=torch.float)
Y_val_t = torch.tensor(Y_val, dtype=torch.float)


print(Y_train_t.dtype)
print(Y_val_t.dtype)

torch.float32
torch.float32
time: 0 ns (started: 2023-12-10 21:04:34 -05:00)


In [9]:
# Defining PCA function
def PCA_function(X_train, X_val, K):
  pca = PCA(n_components=K)
  X_train_PCA = pca.fit_transform(X_train)
  X_val_PCA = pca.fit_transform(X_val)
  return X_train_PCA, X_val_PCA

time: 0 ns (started: 2023-12-10 21:04:34 -05:00)


In [10]:
# Using the PCA function we defined to condense the inputs into 2 features
(X_train_PCA, X_val_PCA) = PCA_function(X_train, X_val, 2)

print("The shape of X_train after PCA is", X_train_PCA.shape)
print("The shape of X_val after PCA is", X_val_PCA.shape)

# Use standard scaling from Sklearn to scale data between - and  for better accuracy.
scale = StandardScaler()
X_train_PCA = scale.fit_transform(X_train_PCA)
X_val_PCA = scale.fit_transform(X_val_PCA)

X_train_PCA_t = torch.tensor(X_train_PCA, dtype=torch.float)
X_val_PCA_t = torch.tensor(X_val_PCA, dtype=torch.float)

print("\nThe shape of the X_train tensor is", X_train_PCA_t.shape)
print("The shape of the X_val tensor is", X_val_PCA_t.shape)

The shape of X_train after PCA is (7987, 2)
The shape of X_val after PCA is (1997, 2)

The shape of the X_train tensor is torch.Size([7987, 2])
The shape of the X_val tensor is torch.Size([1997, 2])
time: 16 ms (started: 2023-12-10 21:04:34 -05:00)


In [11]:
# Defining all the functions needed for training the linear model

def model_linear(t_in, w1, w2, w3, b):
  t_p = ((w3 * t_in ** 3) + (w2 * t_in ** 2) + (w1 * t_in) + b)
  return t_p

def loss_fn(t_p, t_gnd):
  squared_diffs = (t_p - t_gnd)**2
  return squared_diffs.mean()

def training_loop(n_epochs, optimizer, params, train_t_in, val_t_in,
                    train_t_out, val_t_out):
  for epoch in range(1, n_epochs + 1):

    if params.grad is not None:
      params.grad.zero_()

    train_t_p = model_linear(train_t_in, *params)
    train_loss = loss_fn(train_t_p, train_t_out)

    val_t_p = model_linear(val_t_in, *params)
    val_loss = loss_fn(val_t_p, val_t_out)

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch <= 3 or epoch % 500 == 0:
      print(f"Epoch {epoch}, \tTraining loss {train_loss.item():.4f},"
      f" \tValidation loss {val_loss.item():.4f}")

  return params

time: 0 ns (started: 2023-12-10 21:04:34 -05:00)


In [12]:
# Calling the linear model

params = torch.tensor([1.0, 1.0, 1.0, 0.0], requires_grad=True)

learning_rate_1 = 1e-3
optimizer_1 = optim.SGD([params], lr=learning_rate_1)

print('Loss every 500 epochs using learning rate of 1e-3\n')
training_loop(
    n_epochs = 5000,
    optimizer = optimizer_1,
    params = params,
    train_t_in = X_train_PCA_t,
    val_t_in = X_val_PCA_t,
    train_t_out = Y_train_t,
    val_t_out = Y_val_t)

Loss every 500 epochs using learning rate of 1e-3

Epoch 1, 	Training loss 12.1368, 	Validation loss 11.7982
Epoch 2, 	Training loss 11.8755, 	Validation loss 11.5454
Epoch 3, 	Training loss 11.6204, 	Validation loss 11.2984
Epoch 500, 	Training loss 0.4054, 	Validation loss 0.3959
Epoch 1000, 	Training loss 0.3196, 	Validation loss 0.3155
Epoch 1500, 	Training loss 0.2811, 	Validation loss 0.2791
Epoch 2000, 	Training loss 0.2620, 	Validation loss 0.2610
Epoch 2500, 	Training loss 0.2521, 	Validation loss 0.2515
Epoch 3000, 	Training loss 0.2465, 	Validation loss 0.2461
Epoch 3500, 	Training loss 0.2432, 	Validation loss 0.2429
Epoch 4000, 	Training loss 0.2411, 	Validation loss 0.2408
Epoch 4500, 	Training loss 0.2397, 	Validation loss 0.2393
Epoch 5000, 	Training loss 0.2387, 	Validation loss 0.2383


tensor([ 0.1219, -0.0076, -0.0511,  0.3853], requires_grad=True)

time: 2.33 s (started: 2023-12-10 21:04:34 -05:00)


In [13]:
# Getting the predictions based on the training set using the linear model
model_output = model_linear(X_train_PCA_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_np = model_output.detach().numpy() if isinstance(model_output, torch.Tensor) else model_output

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels = np.argmax(model_output_np, axis=1)

# Calculate accuracy, precision, and recall for training set
train_accuracy = accuracy_score(Y_train, predicted_labels)
train_recall = recall_score(Y_train, predicted_labels, average= 'macro', zero_division=1)
train_precision = precision_score(Y_train, predicted_labels, average = 'macro', zero_division=1)
train_F1_score = (2*train_recall*train_precision)/(train_recall+train_precision)

# Getting the predictions based on the validation set using the linear model
model_output_val = model_linear(X_val_PCA_t, *params)
# Converting the prediction tensor into a numpy array to use the sklearn metrics library
model_output_val_np = model_output_val.detach().numpy() if isinstance(model_output_val, torch.Tensor) else model_output_val

# Using argmax to get the class (1 or 0) with the highest probability
predicted_labels_val = np.argmax(model_output_val_np, axis=1)

# Calculate accuracy, precision, and recall for validation set
val_accuracy = accuracy_score(Y_val, predicted_labels_val)
val_precision = precision_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_recall = recall_score(Y_val, predicted_labels_val, average = 'macro', zero_division=1)
val_F1_score = (2*val_recall*val_precision)/(val_recall+val_precision)

# Print the calculated metrics
print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Training Precision: {train_precision:.4f}")
print(f"Training Recall: {train_recall:.4f}")
print(f"Training F1 Score: {train_F1_score:.4f}")

print(f"\nValidation Accuracy: {val_accuracy:.4f}")
print(f"Validation Precision: {val_precision:.4f}")
print(f"Validation Recall: {val_recall:.4f}")
print(f"Validation F1 Score: {val_F1_score:.4f}")


Training Accuracy: 0.4973
Training Precision: 0.4837
Training Recall: 0.4830
Training F1 Score: 0.4833

Validation Accuracy: 0.5333
Validation Precision: 0.5428
Validation Recall: 0.5452
Validation F1 Score: 0.5440
time: 15 ms (started: 2023-12-10 21:04:37 -05:00)


In [14]:
# Sequential Model

model_seq = nn.Sequential(
    nn.Linear(2, 1024),
    nn.Tanh(),
    nn.Linear(1024, 2048),
    nn.Tanh(),
    nn.Linear(2048, 512),
    nn.Tanh(),
    nn.Linear(512, 1)
            )
model_seq

Sequential(
  (0): Linear(in_features=2, out_features=1024, bias=True)
  (1): Tanh()
  (2): Linear(in_features=1024, out_features=2048, bias=True)
  (3): Tanh()
  (4): Linear(in_features=2048, out_features=512, bias=True)
  (5): Tanh()
  (6): Linear(in_features=512, out_features=1, bias=True)
)

time: 31 ms (started: 2023-12-10 21:04:37 -05:00)


In [15]:
sum([p.numel() for p in model_seq.parameters()])

3151873

time: 0 ns (started: 2023-12-10 21:04:37 -05:00)


In [16]:
#Let's define our training loop that will be used

def training_loop_seq(n_epochs, optimizer, model, loss_fn, x_train, x_val,
                  y_train, y_val):
    for epoch in range(1, n_epochs + 1):
        p_train = model(x_train) # <1>
        loss_train = loss_fn(p_train, y_train,)

        p_val = model(x_val) # <1>
        loss_val = loss_fn(p_val, y_val)

        optimizer.zero_grad()
        loss_train.backward() # <2>
        optimizer.step()


        if epoch == 1 or epoch % 10 == 0:
            print(f"Epoch: {epoch}, Training loss: {loss_train.item():.4f},"
                  f" Validation loss: {loss_val.item():.4f}")

time: 0 ns (started: 2023-12-10 21:04:37 -05:00)


In [17]:
#Let's try doing some training

optimizer_1 = optim.Adam(model_seq.parameters(), lr=1e-4)

training_loop_seq(
    n_epochs = 151,
    optimizer = optimizer_1,
    model = model_seq,
    loss_fn = nn.MSELoss(),
    x_train = X_train_PCA_t,
    x_val = X_val_PCA_t,
    y_train = Y_train_t,
    y_val = Y_val_t
)

Epoch: 1, Training loss: 0.3641, Validation loss: 0.3591
Epoch: 10, Training loss: 0.2446, Validation loss: 0.2442
Epoch: 20, Training loss: 0.2359, Validation loss: 0.2342
Epoch: 30, Training loss: 0.2383, Validation loss: 0.2364
Epoch: 40, Training loss: 0.2363, Validation loss: 0.2347
Epoch: 50, Training loss: 0.2354, Validation loss: 0.2341
Epoch: 60, Training loss: 0.2353, Validation loss: 0.2341
Epoch: 70, Training loss: 0.2352, Validation loss: 0.2341
Epoch: 80, Training loss: 0.2352, Validation loss: 0.2340
Epoch: 90, Training loss: 0.2352, Validation loss: 0.2340
Epoch: 100, Training loss: 0.2351, Validation loss: 0.2339
Epoch: 110, Training loss: 0.2351, Validation loss: 0.2339
Epoch: 120, Training loss: 0.2351, Validation loss: 0.2339
Epoch: 130, Training loss: 0.2350, Validation loss: 0.2338
Epoch: 140, Training loss: 0.2350, Validation loss: 0.2338
Epoch: 150, Training loss: 0.2350, Validation loss: 0.2337
time: 1min 43s (started: 2023-12-10 21:04:37 -05:00)


In [18]:
# Function to calculate accuracy
def calculate_accuracy(predictions, targets):
    return (predictions == targets).float().mean().item()

# Function to calculate recall
def calculate_recall(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_negatives = ((predictions == 0) & (targets == 1)).sum().item()
    return true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0

# Function to calculate precision
def calculate_precision(predictions, targets):
    true_positives = ((predictions == 1) & (targets == 1)).sum().item()
    false_positives = ((predictions == 1) & (targets == 0)).sum().item()
    return true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0

time: 0 ns (started: 2023-12-10 21:06:20 -05:00)


In [19]:
# Calculate predictions for the training set
with torch.no_grad():
    train_predictions = torch.round(torch.sigmoid(model_seq(X_train_PCA_t)))

# Calculate accuracy, recall, precision, and F1 Score for the training set
accuracy_train = calculate_accuracy(train_predictions, Y_train_t)
recall_train = calculate_recall(train_predictions, Y_train_t)
precision_train = calculate_precision(train_predictions, Y_train_t)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Calculate predictions for the validation set
with torch.no_grad():
    train_predictions_val = torch.round(torch.sigmoid(model_seq(X_val_PCA_t)))

# Calculate accuracy, recall, precision, and F1 Score for the validation set
accuracy_val = calculate_accuracy(train_predictions_val, Y_val_t)
recall_val = calculate_recall(train_predictions_val, Y_val_t)
precision_val = calculate_precision(train_predictions_val, Y_val_t)
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)

# Print the calculated metrics
print("Training Accuracy: {:.4f}".format(accuracy_train))
print("Training Recall: {:.4f}".format(recall_train))
print("Training Precision: {:.4f}".format(precision_train))
print("Training F1 Score: {:.4f}".format(F1_score_train))

print("\nValidation Accuracy: {:.4f}".format(accuracy_val))
print("Validation Recall: {:.4f}".format(recall_val))
print("Validation Precision: {:.4f}".format(precision_val))
print("Validation F1 Score: {:.4f}".format(F1_score_val))

Training Accuracy: 0.3825
Training Recall: 1.0000
Training Precision: 0.3825
Training F1 Score: 0.5533

Validation Accuracy: 0.3791
Validation Recall: 1.0000
Validation Precision: 0.3791
Validation F1 Score: 0.5497
time: 281 ms (started: 2023-12-10 21:06:20 -05:00)


In [20]:
model_fnn = nn.Sequential(
            nn.Linear(2, 1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 2048),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(2048,1024),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(1024, 512),
            nn.Tanh(),
            nn.Dropout(p=0.5),  # Dropout layer with a dropout probability of 0.5
            nn.Linear(512, 64),
            nn.Tanh(),
            nn.Linear(64, 1)
           )

time: 16 ms (started: 2023-12-10 21:06:20 -05:00)


In [21]:
sum([p.numel() for p in model_fnn.parameters()])

4758145

time: 15 ms (started: 2023-12-10 21:06:20 -05:00)


In [22]:
learning_rate = 1

optimizer = optim.SGD(model_fnn.parameters(), lr=learning_rate)

loss_fn = nn.BCEWithLogitsLoss()

n_epochs = 151

#This is the Training

for epoch in range(n_epochs):
    outputs = model_fnn(X_train_PCA_t)
    loss = loss_fn(outputs, Y_train_t)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:  # Print loss every 10 epochs
      print("Epoch: %d, Loss: %f" % (epoch, float(loss)))

Epoch: 0, Loss: 0.687828
Epoch: 10, Loss: 0.666923
Epoch: 20, Loss: 0.666157
Epoch: 30, Loss: 0.664906
Epoch: 40, Loss: 0.665643
Epoch: 50, Loss: 0.664982
Epoch: 60, Loss: 0.665459
Epoch: 70, Loss: 0.665062
Epoch: 80, Loss: 0.664465
Epoch: 90, Loss: 0.664202
Epoch: 100, Loss: 0.664330
Epoch: 110, Loss: 0.664178
Epoch: 120, Loss: 0.664371
Epoch: 130, Loss: 0.664425
Epoch: 140, Loss: 0.664458
Epoch: 150, Loss: 0.663770
time: 2min 24s (started: 2023-12-10 21:06:20 -05:00)


In [23]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Training Set

correct_train = 0
total_train = 0

with torch.no_grad():
    outputs_train = model_fnn(X_train_PCA_t)
    _, predicted_train = torch.max(outputs_train, dim=1)
    total_train += Y_train_t.size(0)
    correct_train += int((predicted_train == Y_train_t.squeeze()).sum())

accuracy_train = correct_train / total_train
precision_train = precision_score(Y_train_t, predicted_train, average='macro', zero_division=0)
recall_train = recall_score(Y_train_t, predicted_train, average='macro', zero_division=0)
F1_score_train = (2*recall_train*precision_train)/(recall_train+precision_train)

# Print the metrics
print("Training Accuracy: %f" % accuracy_train)
print("Training Precision: %f" % precision_train)
print("Training Recall: %f" % recall_train)
print("Training F1 score: %f" % F1_score_train)

Training Accuracy: 0.617503
Training Precision: 0.308752
Training Recall: 0.500000
Training F1 score: 0.381763
time: 328 ms (started: 2023-12-10 21:08:45 -05:00)


In [24]:
# Checking the Accuracy, Recall, Precision, and F1 Score of the Validation Set

correct_val = 0
total_val = 0

with torch.no_grad():
    outputs_val = model_fnn(X_val_PCA_t)
    _, predicted_val = torch.max(outputs_val, dim=1)
    total_val += Y_val_t.size(0)
    correct_val += int((predicted_val == Y_val_t.squeeze()).sum())

accuracy_val = correct_val / total_val
precision_val = precision_score(Y_val_t, predicted_val, average='macro', zero_division=0)
recall_val = recall_score(Y_val_t, predicted_val, average='macro', zero_division=0)
F1_score_val = (2*recall_val*precision_val)/(recall_val+precision_val)

# Print the metrics
print("Validation Accuracy: %f" % accuracy_val)
print("Training Precision: %f" % precision_val)
print("Training Recall: %f" % recall_val)
print("Training F1 score: %f" % F1_score_val)

Validation Accuracy: 0.620931
Training Precision: 0.310466
Training Recall: 0.500000
Training F1 score: 0.383071
time: 78 ms (started: 2023-12-10 21:08:45 -05:00)
